In [124]:
%reload_ext autoreload
%autoreload 2

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", os.pardir))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import test_config, setup
from src import pipeline
from src.storage import retrieve


config_data = test_config.get_config_data()
scenerio_data = config_data["test"]["scenerios"][0]

setup.initialize_data_stores(config_data)

In [ ]:
pipeline.backfill_stm(config_data, scenerio_data)

In [ ]:
bot_datas = {bot["full_name"]: bot for bot in scenerio_data["users"]["bots"]}
print([key for key in bot_datas.keys()])

In [ ]:
bot_data = bot_datas["Juan Tana Mera"]
for question in config_data["create"]["questions"][:1]:
    pipeline.answer_question(config_data, scenerio_data, bot_data, question)

In [ ]:
message = retrieve.get_latest_event(config_data, scenerio_data)

bot_data = bot_datas["Juan Tana Mera"]  # bot you want to respond
response = pipeline.chat(config_data, scenerio_data, bot_data, message)
pipeline.update_stm(config_data, scenerio_data, bot_data, response)

In [ ]:
message = retrieve.get_latest_event(config_data, scenerio_data)

pipeline.update_mtm(config_data, scenerio_data, message)
pipeline.update_ltm(config_data, scenerio_data, message)

In [ ]:
# print all the memory stores?
print(config_data["summarize"]["mtm_template"])